<a href="https://colab.research.google.com/github/seymasel/CS550/blob/main/2022_hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PART 1 LAMPORT'S LOGICAL CLOCKS


REFERENCE: https://towardsdatascience.com/understanding-lamport-timestamps-with-pythons-multiprocessing-library-12a6427881c6

In [ ]:
from multiprocessing import Process, Pipe
from os import getpid
from datetime import datetime
from time import sleep

In [ ]:
def local_time(counter):
    return ' (LAMPORT_TIME={}, LOCAL_TIME={})'.format(counter,datetime.now())

In [ ]:
def calc_recv_timestamp(recv_time_stamp, counter):
    return max(recv_time_stamp, counter) + 1

In [ ]:
def event(pid, counter):
    counter += 1
    print('Something happened in {} !'.\
          format(pid) + local_time(counter) + '\r\n')
    return counter

In [ ]:
def send_message(pipe, pid, counter):
    counter += 1
    pipe.send(('Empty shell', counter))
    print('Message sent from ' + str(pid) + local_time(counter) + '\r\n')
    return counter

In [ ]:
def recv_message(pipe, pid, counter):
    message, timestamp = pipe.recv()
    counter = calc_recv_timestamp(timestamp, counter)
    print('Message received at ' + str(pid)  + local_time(counter) + '\r\n')
    return counter

In [ ]:
def process_one(pipe12,pipe13):
    pid = getpid()
    counter = 0
    counter = event(pid, counter)
    counter = send_message(pipe12, pid, counter)
    counter = recv_message(pipe12, pid, counter)
    counter = send_message(pipe13, pid, counter)
    counter = recv_message(pipe13, pid, counter)
    counter = send_message(pipe12, pid, counter)
    counter  = event(pid, counter)

def process_two(pipe21):
    pid = getpid()
    counter = 0
    counter = send_message(pipe21, pid, counter)
    counter = recv_message(pipe21, pid, counter)
    counter = recv_message(pipe21, pid, counter)



def process_three(pipe31):
    pid = getpid()
    counter = 0
    counter  = event(pid, counter)    
    counter = send_message(pipe31, pid, counter)
    counter = recv_message(pipe31, pid, counter)


In [ ]:
if __name__ == '__main__':
    oneandtwo, twoandone = Pipe()
    oneandthree, threeandone = Pipe()

    process1 = Process(target=process_one, 
                       args=(oneandtwo,oneandthree,))
    process2 = Process(target=process_two,
                       args=(twoandone,))
    process3 = Process(target=process_three, 
                       args=(threeandone,))

    process1.start()
    process2.start()
    process3.start()

    process1.join()
    process2.join()
    process3.join()

Something happened in 2295 ! (LAMPORT_TIME=1, LOCAL_TIME=2022-12-09 12:28:11.205429)
Message sent from 2298 (LAMPORT_TIME=1, LOCAL_TIME=2022-12-09 12:28:11.211888)


Message received at 2298 (LAMPORT_TIME=3, LOCAL_TIME=2022-12-09 12:28:11.240428)
Message sent from 2295 (LAMPORT_TIME=2, LOCAL_TIME=2022-12-09 12:28:11.240004)
Something happened in 2301 ! (LAMPORT_TIME=1, LOCAL_TIME=2022-12-09 12:28:11.233657)



Message received at 2295 (LAMPORT_TIME=3, LOCAL_TIME=2022-12-09 12:28:11.253911)
Message sent from 2301 (LAMPORT_TIME=2, LOCAL_TIME=2022-12-09 12:28:11.258132)


Message sent from 2295 (LAMPORT_TIME=4, LOCAL_TIME=2022-12-09 12:28:11.264433)
Message received at 2301 (LAMPORT_TIME=5, LOCAL_TIME=2022-12-09 12:28:11.268628)


Message received at 2295 (LAMPORT_TIME=5, LOCAL_TIME=2022-12-09 12:28:11.276612)

Message sent from 2295 (LAMPORT_TIME=6, LOCAL_TIME=2022-12-09 12:28:11.290932)
Message received at 2298 (LAMPORT_TIME=7, LOCAL_TIME=2022-12-09 12:28:11.291416)

Something happened 

PART 2 VECTORS CLOCKS


In [ ]:
def calc_recv_timestamp(recv_time_stamp, counter):
    for id  in range(len(counter)):
        counter[id] = max(recv_time_stamp[id], counter[id])
    return counter

In [ ]:
def event(pid, counter):
    counter[pid] += 1
    print('Something happened in {} !'.\
          format(pid) + local_time(counter)  + '\r\n')
    return counter

In [ ]:
def send_message(pipe, pid, counter):
    counter[pid] += 1
    pipe.send(('Empty shell', counter))
    print('Message sent from ' + str(pid) + local_time(counter) + '\r\n')
    return counter

In [ ]:
def recv_message(pipe, pid, counter):
    counter[pid] += 1
    message, timestamp = pipe.recv()
    counter = calc_recv_timestamp(timestamp, counter)
    print('Message received at ' + str(pid)  + local_time(counter) + '\r\n')
    return counter

In [ ]:
def process_one(pipe12,pipe13):
    pid = 0
    counter = [0,0,0]
    counter = event(pid, counter)
    counter = send_message(pipe12, pid, counter)
    counter = recv_message(pipe12, pid, counter)
    counter = send_message(pipe13, pid, counter)
    counter = recv_message(pipe13, pid, counter)
    counter = send_message(pipe12, pid, counter)
    counter  = event(pid, counter)

def process_two(pipe21):
    pid = 1
    counter = [0,0,0]
    counter = send_message(pipe21, pid, counter)
    counter = recv_message(pipe21, pid, counter)
    counter = recv_message(pipe21, pid, counter)



def process_three(pipe31):
    pid = 2
    counter = [0,0,0]
    counter  = event(pid, counter)    
    counter = send_message(pipe31, pid, counter)
    counter = recv_message(pipe31, pid, counter)

In [ ]:
if __name__ == '__main__':
    oneandtwo, twoandone = Pipe()
    oneandthree, threeandone = Pipe()

    process1 = Process(target=process_one, args=(oneandtwo,oneandthree,))
    process2 = Process(target=process_two,args=(twoandone,))
    process3 = Process(target=process_three, args=(threeandone,))

    process1.start()
    process2.start()
    process3.start()

    process1.join()
    process2.join()
    process3.join()

Something happened in 0 ! (LAMPORT_TIME=[1, 0, 0], LOCAL_TIME=2022-12-09 12:28:11.421191)
Message sent from 1 (LAMPORT_TIME=[0, 1, 0], LOCAL_TIME=2022-12-09 12:28:11.433736)

Message sent from 0 (LAMPORT_TIME=[2, 0, 0], LOCAL_TIME=2022-12-09 12:28:11.450052)
Something happened in 2 ! (LAMPORT_TIME=[0, 0, 1], LOCAL_TIME=2022-12-09 12:28:11.440667)



Message received at 0 (LAMPORT_TIME=[3, 1, 0], LOCAL_TIME=2022-12-09 12:28:11.466976)
Message received at 1 (LAMPORT_TIME=[2, 2, 0], LOCAL_TIME=2022-12-09 12:28:11.463365)

Message sent from 2 (LAMPORT_TIME=[0, 0, 2], LOCAL_TIME=2022-12-09 12:28:11.472454)

Message sent from 0 (LAMPORT_TIME=[4, 1, 0], LOCAL_TIME=2022-12-09 12:28:11.475080)

Message received at 2 (LAMPORT_TIME=[4, 1, 3], LOCAL_TIME=2022-12-09 12:28:11.486631)

Message received at 0 (LAMPORT_TIME=[5, 1, 2], LOCAL_TIME=2022-12-09 12:28:11.499718)


Message received at 1 (LAMPORT_TIME=[6, 3, 2], LOCAL_TIME=2022-12-09 12:28:11.514398)
Message sent from 0 (LAMPORT_TIME=[6, 1, 2],